# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

**The Cournot Game**

Imports and set magics:

In [4]:
import numpy as np
import warnings 
import scipy.optimize as optimize
import plotly.express as px
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Layout

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

In this project I will analyse the Cournot game. I will here look at a market with n firms and see what happens with the equrelibrium when adding more firms in the equation. This means that the firms will choose a quantity q and the economy will end in some nash equilibrium.

I will consider an economy where I have to identical firms that each produce a good. these firms will then simultanius choose a quantity $q{_i}$ which is split into $q{_1}$ and $q{_2}$. this means that the total quanity of goods on the market equals
 $q=q{_1}+q{_2}+...+q{_n}$
The output for the firms in my economy is then
$q{2}=q+q{_1}$
$q{1}=q-q{_2}$

The marginal cost for the firms will be $c{_i}$ where with $<=c{_i}<a$ where a is just some constant that will indicate where the two firms intercede.
the inverse demand function will be defined as

$p(q)$

each firm want to maximise thier profits givin the other firms output therefore I can write the firms profit function as

$\PI{_1}(q{_1},q{2})=(a-bq{_1}-bq{_2}-c)q{_1} $
$\PI{_2}(q{_1},q{2})=(a-bq{_1}-bq{_2}-c)q{_2} $

The firms will be assumed to maximize profits by taking the first order I will find just that.
$
\frac{d \Pi_1}{d q_1} = a-bq{_1}-bq{_2}-c = 0
$
$
\frac{d \Pi_2}{d q_2} = a-bq{_1}-bq{_2}-c = 0
$
the best response (BR) for both firms will then be. 
$
q{_1}=\frac{a-c}{2b}-\frac{1}{2}*q{_2}
$
$
q{_2}=\frac{a-c}{2b}-\frac{1}{2}*q{_1}
$

and this is the model i will test for.

**Writing the Cournot economy**

In [9]:
#first I will define the different functions for the firms

# define the demand function and the cost function
def p(q1, q2, a, b):
    return a - b*(q1 + q2)

def cost(q, c):
    return q * c 

# define the profit function for the firms
def firm_profit_1(q1, q2, a, b, c):
    return p(q1, q2, a, b) * q1 - cost(q1, c)

def firm_profit_2(q1, q2, a, b, c):
    return p(q1, q2, a, b) * q2 - cost(q2, c)

# define the best response function for the firm
def BR_1(q2, a, b, c):
    Q1 = optimize.minimize(lambda q0: -firm_profit_1(q0, q2, a, b, c), [0]).x[0]
    return max(Q1, 0)

def BR_2(q1, a, b, c):
    Q2 = optimize.minimize(lambda q0: -firm_profit_2(q1, q0, a, b, c), [0]).x[0]
    return max(Q2, 0)

# define the conditions for the Nash equilibrium
def conditions(q,a, b, c ):
    u = q[0] - BR_1(q[1],a, b, c)
    y = q[1] - BR_2(q[0], a, b, c)
    return [u, y]

# set the initial guess and parameters
initial_guess = [1,1]
a = 20
b = 1
c = 2

# use the fsolve function to find the Nash equilibrium
solver = optimize.fsolve(conditions, initial_guess, args=(a, b, c))
q_star = solver[0]

# print the NE
print(f'The Cournot/Nash Equilibirum is : \n q_1^* = q_2^* = {q_star:.2f} \n')

# Calculate and print the price and profit at the NE
p_star = p(q_star, q_star, a, b)
print(f'The price at the Nash equilibrium is: \n p = {p_star:.2f}')

pi_star = firm_profit_1(q_star, q_star, a, b, c)
print(f'The profits at the Nash equilibrium is: \n pi = {pi_star:.2f}')

The Cournot/Nash Equilibirum is : 
 q_1^* = q_2^* = 6.00 

The price at the Nash equilibrium is: 
 p = 8.00
The profits at the Nash equilibrium is: 
 pi = 36.00


Here it can be seen that the firms will both produce 6 units of the goods. firm one will produce to a price of 8 and get a profit of 36. firm 2 will be identical given that the two firms have identical profit functions.
I will next plot the two firms in a plot to see the output graphicly

In [12]:
# define production levels for firm 1
production_level_f1 = np.arange(0, 10, 0.1)

# calculate production levels for the firms using the BR function
production_level_f2 = np.vectorize(BR_2)(production_level_f1, a, b, c)
production_level_f1_2 = np.vectorize(BR_1)(production_level_f2, a, b, c)

# create the scatter plot
fig = px.scatter()

# add lines for firm 1 and 2
fig.add_scatter(x=production_level_f2, y=production_level_f1, mode='lines', name='Firm 1', line=dict(color='red'))
fig.add_scatter(x=production_level_f1, y=production_level_f2, mode='lines', name='Firm 2', line=dict(color='blue'))

# add titles
fig.update_layout(title='BR for both firms')
fig.update_xaxes(title='Firm 1 output')
fig.update_yaxes(title='Firm 2 output')

fig.show()

Here it can be seen that both firms will choose to produce 6 units of the goode. Two interesting things to note. first when only one firm produces the output in the economy will be 9 instead of 12. It is better for the economy the more firms are competing against each other. The second thing is that for this to work it is important that the economy is a cornot and not a stackelberg sinse that would also produce an inefficient amount of firm 1 was the price leader.

**New paremeter values**

Now I will update the model so that different paremter values can be used to see how the firms change the NE in the economy.

In [16]:
N = 2

# define quantity and price
def cournot(N, a, b, c):
    q = (a - b*N - c*N) / (2*N)  
    p = a - b*q - N*q 
    return p, q

# create plot 
def cournot_model(a, b, c):
    p, q = cournot(N, a, b, c) 
    fig, ax = plt.subplots()
    x = np.linspace(0, (a-b*N)/N, 100)  
    ax.plot(x, a - b*x - N*x, lw=2, label='Demand')  
    ax.plot(q, p, 'ro', label='Equilibrium')  
    ax.set_xlabel('Quantity')
    ax.set_ylabel('Price')
    ax.set_xlim([0, (a-b*N)/N])  
    ax.set_ylim([0, a])  
    ax.legend(loc='best')
    plt.title(f'{N} firms, a={a}, b={b}, c={c}')  
    plt.show()

# create sliders for a, b and c
a_slider = FloatSlider(min=0, max=20, step=0.5, value=10, description='a:', layout=Layout(width='50%'))
b_slider = FloatSlider(min=0, max=5, step=0.1, value=1, description='b:', layout=Layout(width='50%'))
c_slider = FloatSlider(min=0, max=5, step=0.1, value=1, description='c:', layout=Layout(width='50%'))
# use the sliders to interact with the cournot_model function
interact(cournot_model, a=a_slider, b=b_slider, c=c_slider,)

interactive(children=(FloatSlider(value=10.0, description='a:', layout=Layout(width='50%'), max=20.0, step=0.5…

<function __main__.cournot_model(a, b, c)>

a as desribed above it the intersection the higer a is the more demand for the goode exist in the economy.
b is the slope of the graph. Which means than when b increases the dependency of the other price for the opposing firm. a high value of b would lead to lower amounts of goods produced in the economy for a higher price than before.
c will increase the marginal cost of producing each goode. the higher c will make it so demand will wall given that both firms will have to increae prices.

**More firms**

I will now expand the economy so that it is able to hold more three firms in the market which should show the point of more firms increases competion and makes the economy healthier.

In [21]:
# define the demand function and cost function
def p(q1, q2, q3, a, b):
    return a - b*(q1 + q2 + q3)

def cost(q, c):
    return q * c 

# define the profit function for firm 1, firm 2 and firm 3
def firm_profit_1(q1, q2, q3, a, b, c):
    return p(q1, q2, q3, a, b) * q1 - cost(q1, c)

def firm_profit_2(q1, q2, q3, a, b, c):
    return p(q1, q2, q3, a, b) * q2 - cost(q2, c)

def firm_profit_3(q1, q2, q3, a, b, c):
    return p(q1, q2, q3, a, b) * q3 - cost(q3, c)

# Define the best response function for firm 1, firm 2 and firm 3
def BR_1(q2, q3, a, b, c):
    Q1 = optimize.minimize(lambda q0: -firm_profit_1(q0, q2, q3, a, b, c), [0]).x[0]
    return Q1

def BR_2(q1, q3, a, b, c):
    Q2 = optimize.minimize(lambda q0: -firm_profit_2(q1, q0, q3, a, b, c), [0]).x[0]
    return Q2

def BR_3(q1, q2, a, b, c):
    Q3 = optimize.minimize(lambda q0: -firm_profit_3(q1, q2, q0, a, b, c), [0]).x[0]
    return Q3

# define the conditions for NE
def conditions(q,a, b, c ):
    u = q[0] - BR_1(q[1], q[2], a, b, c) # condition for firm 1
    y = q[1] - BR_2(q[0], q[2], a, b, c) # condition for firm 2
    z = q[2] - BR_3(q[0], q[1], a, b, c) # condition for firm 3
    return [u, y, z]

# set the initial guess
initial_guess = [1,1,1]
a = 20
b = 1
c = 2

# solve for NE
solver = optimize.fsolve(conditions,initial_guess, args = (a, b, c))
q_star = solver
print(f'The Cournot/Nash Equilibirum is : \n q_1^* = {q_star[0]:.2f}, q_2^* = {q_star[1]:.2f}, q_3^* = {q_star[2]:.2f}\n')

# Calculate and print the price and profit at the NE
p_star = p(q_star[0], q_star[1], q_star[2], a, b)
print(f'The price at the Nash equilibrium is: \n p = {p_star:.2f}')

pi_star_i = firm_profit_1(q_star[0], q_star[1], q_star[2], a, b, c)
print(f'The profits for firm i at the Nash equilibrium is: \n pi_i = {pi_star_i:.2f}')

The Cournot/Nash Equilibirum is : 
 q_1^* = 4.50, q_2^* = 4.50, q_3^* = 4.50

The price at the Nash equilibrium is: 
 p = 6.50
The profits for firm i at the Nash equilibrium is: 
 pi_i = 20.25


It can from these results be seen that the firms will now produce a higher amount of goods in this economy. they will produce 13,5 instead of 12. this means that more firms will increase the output in the economy.
the price will also fall the more firms enter the market. so if an infinite amount of firms entered the NE price will go towards the cost and the profits on the market will go towards 0. which is the condition for perfect competion in the economy.


# Conclusion

In this project I have looked at the cournot game. and what happens once different variables are cahnged in the market.
I have here found that increasing the competetiveness in the econonmy will help to lower prices and increase demand. and I have looked at what happens if the market is just split between two firms and if one firm had a monopoly on the market.